In [ ]:
import numpy as np
import rasterio as rio
from rasterio.transform import from_origin
from tqdm import tqdm
import timeit


In [ ]:

# Import custom functions
def to_numpy2(transform):
    return np.array([
        transform.a, transform.b, transform.c,
        transform.d, transform.e, transform.f,
        0, 0, 1
    ], dtype='float64').reshape((3, 3))

def xy_np(transform, rows, cols, offset='center'):
    if isinstance(rows, int) and isinstance(cols, int):
        pts = np.array([[rows, cols, 1]]).T
    else:
        assert len(rows) == len(cols)
        pts = np.ones((3, len(rows)), dtype=int)
        pts[0] = rows
        pts[1] = cols

    if offset == 'center':
        coff, roff = (0.5, 0.5)
    elif offset == 'ul':
        coff, roff = (0, 0)
    elif offset == 'ur':
        coff, roff = (1, 0)
    elif offset == 'll':
        coff, roff = (0, 1)
    elif offset == 'lr':
        coff, roff = (1, 1)
    else:
        raise ValueError("Invalid offset")

    _transnp = to_numpy2(transform)
    _translt = to_numpy2(transform.translation(coff, roff))
    locs = _transnp @ _translt @ pts
    return locs[0].tolist(), locs[1].tolist()

# Original Code Function (Fixed)
def original_code(transform, rows_flat, cols_flat):
    res_dem = [
        (transform * (col, row)) for row, col in tqdm(zip(rows_flat, cols_flat), total=len(rows_flat), desc="Original Code")
    ]
    x_coords, y_coords = zip(*res_dem)
    return np.array(x_coords), np.array(y_coords)

# Create a synthetic DEM-like array
width, height = 1000, 1000  # Adjust size for testing
dem_array = np.random.rand(height, width)

# Define an affine transformation
transform = from_origin(0, 10000, 1, -1)  # Origin (0, 10000), pixel size 1x1

# Create a meshgrid of row and column indices
rows, cols = np.indices(dem_array.shape)
rows_flat = rows.ravel()
cols_flat = cols.ravel()

# Benchmarking with timeit
print("\nRunning Benchmarks...")

rasterio_time = timeit.timeit(
    stmt="rio.transform.xy(transform, rows, cols, offset='center', precision=1)",
    globals=globals(),
    number=10
) / 10

custom_time = timeit.timeit(
    stmt="xy_np(transform, rows_flat, cols_flat, offset='center')",
    globals=globals(),
    number=10
) / 10

original_time = timeit.timeit(
    stmt="original_code(transform, rows_flat, cols_flat)",
    globals=globals(),
    number=10
) / 10

# Results
print("\nBenchmark Results (average over 10 runs):")
print(f"Rasterio transform.xy time: {rasterio_time:.4f} seconds")
print(f"Custom xy_np time: {custom_time:.4f} seconds")
print(f"Original code time: {original_time:.4f} seconds")

# Validation of Results
print("\nValidating Results...")
x_coords_rasterio, y_coords_rasterio = rio.transform.xy(transform, rows, cols, offset='center', precision=1)
x_coords_rasterio = np.array(x_coords_rasterio).ravel()
y_coords_rasterio = np.array(y_coords_rasterio).ravel()

x_coords_custom, y_coords_custom = xy_np(transform, rows_flat, cols_flat, offset='center')
x_coords_original, y_coords_original = original_code(transform, rows_flat, cols_flat)

x_diff_custom = np.max(np.abs(x_coords_rasterio - x_coords_custom))
y_diff_custom = np.max(np.abs(y_coords_rasterio - y_coords_custom))
x_diff_original = np.max(np.abs(x_coords_rasterio - x_coords_original))
y_diff_original = np.max(np.abs(y_coords_rasterio - y_coords_original))

print(f"Custom function - Max X difference: {x_diff_custom}, Max Y difference: {y_diff_custom}")
print(f"Original code - Max X difference: {x_diff_original}, Max Y difference: {y_diff_original}")


# **Conclusion and Motivation for Performance Improvement**

## **Results Analysis**

### **Performance Benchmark Results:**
| **Method**              | **Execution Time (average)** | **Observations**                                |
|--------------------------|------------------------------|------------------------------------------------|
| **Rasterio `transform.xy`** | **0.0287 seconds**          | Fastest method, optimized for vectorized operations. |
| **Custom `xy_np`**        | **0.0401 seconds**          | Slightly slower than Rasterio, but efficient for custom implementation. |
| **Original Code**         | **4.2850 seconds**          | Significantly slower, using a Python loop for transformations. |

### **Accuracy Validation:**
| **Method**              | **Max X Difference** | **Max Y Difference** | **Observations**                                   |
|--------------------------|----------------------|----------------------|---------------------------------------------------|
| **Rasterio `transform.xy`** | -                    | -                    | Used as the baseline for accuracy comparison.     |
| **Custom `xy_np`**        | **999.0**            | **999.0**            | Significant deviation, needs correction for precise results. |
| **Original Code**         | **0.5**              | **0.5**              | Matches closely with Rasterio, indicating high accuracy. |

---

## **Conclusions**

1. **Performance:**
   - Rasterio's `transform.xy` is the fastest and most efficient due to its highly optimized C implementation, handling large datasets seamlessly.
   - The custom `xy_np` function, while slightly slower than Rasterio, demonstrates that matrix-based transformations are more efficient than Python loops.
   - The original code is the slowest due to its reliance on Python loops, which are inherently slower for large-scale operations.

2. **Accuracy:**
   - The custom `xy_np` function has a significant deviation in results (`999.0` difference), indicating potential issues with handling offsets or transformations.
   - The original code closely matches Rasterio's results, validating its correctness but highlighting its inefficiency.

---

## **Motivation for Performance Improvement**

1. **Scalability for Large Datasets:**
   - In real-world applications like UAV image processing or geospatial analysis, datasets often contain millions of pixels. Improving performance directly impacts the feasibility of processing such large data in a reasonable time frame.

2. **Importance of Accuracy:**
   - Accurate coordinate transformation is critical for applications like mapping, modeling, and spatial analysis. Rasterio provides a robust and precise baseline, while the custom implementation needs refinement to ensure correctness.

---
